## Using ak_requests

In [1]:
import common as ak
import materials
import numpy as np
from scipy.optimize import linprog

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US", local=True)
stage_dict = ak.get_stage_dict(lang="zh_CN", local=True)
craft_dict = ak.get_craft_dict(lang="zh_CN", local=True)
psdf = ak.get_pengstats_df(server="CN")

Reading from local file
Reading from remote
Reading from local file


In [3]:
stage_san_cost, stage_names, stage_names_rev, stage_lmd_dict = ak.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = ak.get_event_info(stage_names)

In [4]:
material_ids = ak.get_material_ids(item_names_rev, materials.MATERIAL_NAMES)
n_mats_all = len(material_ids)

In [5]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = ak.get_main_and_perm_stage_ids(psdf)
stage_ids = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

stage_san_all = ak.get_san_cost(stage_ids, stage_san_cost)
n_stages_all = len(stage_ids)

In [6]:
craft_matrix, subprod_matrix = ak.get_craft_matrix(craft_dict, material_ids, item_names_rev)

In [7]:
drop_matrix_all, coeff_matrix_all = ak.get_drop_matrix(psdf, stage_ids, material_ids, stage_lmd_dict)

In [8]:
drop_matrix, stage_san = ak.filter_drop_matrix(drop_matrix_all, stage_san_all)

In [9]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix, subprod_matrix, byprod_rate)

In [10]:
coeff = np.sum(drop_matrix, axis=0)
res = linprog(-coeff,
              A_ub=drop_matrix, b_ub=stage_san,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(material_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.3f}".format(item_names[d], msv[i]))

True
LMD: 0.003
Skill Summary - 1: 1.585
Skill Summary - 2: 4.029
Skill Summary - 3: 10.244
Orirock: 1.352
Orirock Cube: 3.984
Orirock Cluster: 19.410
Orirock Concentration: 73.736
Damaged Device: 3.288
Device: 9.792
Integrated Device: 38.659
Optimized Device: 102.628
Ester: 1.991
Polyester: 5.899
Polyester Pack: 23.087
Polyester Lump: 94.148
Sugar Substitute: 2.015
Sugar: 5.971
Sugar Pack: 23.374
Sugar Lump: 98.992
Oriron Shard: 2.413
Oriron: 7.165
Oriron Cluster: 28.153
Oriron Block: 114.146
Diketon: 2.444
Polyketon: 7.259
Aketon: 28.527
Keton Colloid: 104.519
Loxic Kohl: 23.353
White Horse Kohl: 77.556
Manganese Ore: 27.997
Manganese Trihydrate: 98.528
Grindstone: 29.055
Grindstone Pentahydrate: 91.961
RMA70-12: 34.735
RMA70-24: 98.178
Polymerization Preparation: 277.170
Bipolar Nanoflake: 242.510
D32 Steel: 273.436
Coagulating Gel: 31.029
Polymerized Gel: 82.127
Incandescent Alloy: 26.850
Incandescent Alloy Block: 90.657
Crystalline Component: 22.014
Crystalline Circuit: 98.002
Cry

In [11]:
ak.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


In [17]:
ak.print_stage_drops("R8-11", drop_matrix, stage_ids, material_ids, stage_names_rev, item_names)

LMD: 252
Crystalline Component: 52.80%
Orirock Cube: 36.71%
Orirock: 21.39%
Device: 13.95%
Damaged Device: 7.94%
Orirock Cluster: 3.27%
Crystalline Circuit: 2.68%
Loxic Kohl: 2.67%
Coagulating Gel: 2.08%
Integrated Device: 1.82%


In [13]:
print(stage_names_rev["LS-5"])

wk_kc_5


In [14]:
print(item_names_rev["Pure Gold"])

3003


In [30]:
event_ids = ak.get_event_ids("WR", psdf, event_names_rev)
event_stage_san = ak.get_san_cost(event_ids, stage_san_cost)
event_drop_matrix, _ = ak.get_drop_matrix(psdf, event_ids, material_ids, stage_lmd_dict)
main_drops = ak.get_main_drops(event_drop_matrix, msv, material_ids)
#sanity_return = np.matmul(event_drop_matrix, msv)
stack = np.stack([event_ids, ak.get_stage_efficiency(event_drop_matrix, msv, event_stage_san), main_drops]).T
for a, b, c in stack:
    print("{}: {:.4} {}".format(stage_names[a], float(b), item_names[c]))

WR-1: 0.5894 Sugar Substitute
WR-1 (permanent): 0.5608 Polyester
WR-1 (rerun): 0.5862 Sugar Substitute
WR-2: 0.554 Ester
WR-2 (permanent): 0.6 Polyketon
WR-2 (rerun): 0.5455 Ester
WR-3: 0.5671 Oriron Shard
WR-3 (rerun): 0.5597 Oriron Shard
WR-4: 0.8674 Polyester
WR-4 (rerun): 0.8392 Polyester
WR-5: 0.8797 Polyketon
WR-5 (rerun): 0.7666 Polyketon
WR-6: 0.7747 Orirock Cube
WR-6 (rerun): 0.7326 Orirock Cube
WR-7: 0.8565 Oriron
WR-7 (rerun): 0.8128 Oriron
WR-8: 1.102 Incandescent Alloy
WR-8 (permanent): 0.9275 Incandescent Alloy
WR-8 (rerun): 1.049 Incandescent Alloy
WR-9: 1.096 Polyester Pack
WR-9 (permanent): 0.5963 Polyester Pack
WR-9 (rerun): 1.062 Polyester Pack
WR-10: 1.041 Orirock Cluster
WR-10 (permanent): 0.9341 Orirock Cluster
WR-10 (rerun): 1.01 Orirock Cluster
